In [33]:

from __future__ import print_function
import pandas as pd
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'Data/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

def print_data():
    #print(encoder_input_data, decoder_input_data, decoder_target_data)
    print(encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape)

print_data()

Train on 240 samples, validate on 60 samples
Epoch 1/10


 64/240 [=======>......................] - ETA: 26s - loss: 2.2799

128/240 [===============>..............] - ETA: 10s - loss: 2.2035

192/240 [=======================>......] - ETA: 3s - loss: 2.1259 

240/240 [==============================] - 17s 69ms/step - loss: 2.1109 - val_loss: 2.1090


...

Epoch 10/10


 64/240 [=======>......................] - ETA: 5s - loss: 1.7897

128/240 [===============>..............] - ETA: 3s - loss: 1.7974

192/240 [=======================>......] - ETA: 1s - loss: 1.8405

240/240 [==============================] - 8s 32ms/step - loss: 1.8466 - val_loss: 1.8450


In [34]:
from sklearn.preprocessing import MinMaxScaler

n_features = 4 
labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
labels = labels.drop(columns="Unnamed: 0") 
labels = np.array(labels)
max_len = 0


def prepare_training(path, n_runs):
    def get_max_len(sequence_list):
        m_len = 0
        for seq in sequence_list:
            if len(seq) > m_len:
                m_len = len(seq)
        max_len = m_len
        print(max_len)
        return m_len
    
    
    def construct_matrix(sequence_list):
        max_len = get_max_len(sequence_list)
        train_matrix = np.zeros(shape=(n_runs, max_len, n_features))
        for index,run in enumerate(sequence_list):
            train_matrix[index, :len(run)] = run
        print(train_matrix.shape)
        return train_matrix
        
        
    def stadard_sequences(sequences):
        for i, seq in enumerate(sequences):
            sequences[i] = MinMaxScaler(feature_range=[0, 1]).fit_transform(seq)
        return sequences       
    
    def read_sequences():
        run_list_mix = []
        for index in range(n_runs):
            run_csv = pd.read_csv(path+str(index))
            run_csv = run_csv.drop(columns=['Unnamed: 0'])
            run_list_mix.append(run_csv)
        stand_sequences = stadard_sequences(run_list_mix)
        padded_matrix = construct_matrix(stand_sequences)
        return padded_matrix
    
    return read_sequences()
    

train_matrix = prepare_training("Mix_sequences_var_length/run^", n_runs=300) 


Train on 240 samples, validate on 60 samples
Epoch 1/10


 64/240 [=======>......................] - ETA: 26s - loss: 2.2799

128/240 [===============>..............] - ETA: 10s - loss: 2.2035

192/240 [=======================>......] - ETA: 3s - loss: 2.1259 

240/240 [==============================] - 17s 69ms/step - loss: 2.1109 - val_loss: 2.1090


...

Epoch 10/10


 64/240 [=======>......................] - ETA: 5s - loss: 1.7897

128/240 [===============>..............] - ETA: 3s - loss: 1.7974

192/240 [=======================>......] - ETA: 1s - loss: 1.8405

240/240 [==============================] - 8s 32ms/step - loss: 1.8466 - val_loss: 1.8450


In [35]:

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Conv1D
filters = 50


# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 4))
    
encoder = LSTM(latent_dim, return_state=True)
x = encoder_inputs
for i in range(3):
     x = Conv1D(filters=filters,
                   kernel_size=2,
                   strides=2,
                   padding='same')(x)
     
encoder_outputs, state_h, state_c = encoder(x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

encoder = Model(encoder_inputs, encoder_states)
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, 4))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(4, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()


Train on 240 samples, validate on 60 samples
Epoch 1/10


 64/240 [=======>......................] - ETA: 26s - loss: 2.2799

128/240 [===============>..............] - ETA: 10s - loss: 2.2035

192/240 [=======================>......] - ETA: 3s - loss: 2.1259 

240/240 [==============================] - 17s 69ms/step - loss: 2.1109 - val_loss: 2.1090


...

Epoch 10/10


 64/240 [=======>......................] - ETA: 5s - loss: 1.7897

128/240 [===============>..............] - ETA: 3s - loss: 1.7974

192/240 [=======================>......] - ETA: 1s - loss: 1.8405

240/240 [==============================] - 8s 32ms/step - loss: 1.8466 - val_loss: 1.8450


In [36]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([train_matrix, train_matrix], train_matrix,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.2)


Train on 240 samples, validate on 60 samples
Epoch 1/10


 64/240 [=======>......................] - ETA: 26s - loss: 2.2799

128/240 [===============>..............] - ETA: 10s - loss: 2.2035

192/240 [=======================>......] - ETA: 3s - loss: 2.1259 

240/240 [==============================] - 17s 69ms/step - loss: 2.1109 - val_loss: 2.1090


...

Epoch 10/10


 64/240 [=======>......................] - ETA: 5s - loss: 1.7897

128/240 [===============>..............] - ETA: 3s - loss: 1.7974

192/240 [=======================>......] - ETA: 1s - loss: 1.8405

240/240 [==============================] - 8s 32ms/step - loss: 1.8466 - val_loss: 1.8450


In [37]:
from sklearn.decomposition import PCA

encodings = encoder.predict(train_matrix)
enc_mean, enc_var, z_enc = encodings[0], encodings[1], encodings[0]
print(enc_mean.shape, enc_var.shape, z_enc.shape)


def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg



masks = [return_mask(num, np.array(labels))[:, 0] for num in range(0, 9)]


(300, 256) (300, 256) (300, 256)


In [38]:
from mpl_toolkits.mplot3d import Axes3D  
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K

def plot_pca(title, i): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])

    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask])

        #break
    
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()


enc_list = [enc_mean, enc_var, z_enc]
titles = ["MEAN","LOG_VAR","SAMPLED"]
for i,enc in enumerate(enc_list):
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc) 
    pca = PCA(3)
    principalComponents = pca.fit_transform(enc_input)
    print(principalComponents.shape)
    print(pca.explained_variance_ratio_)
    plot_pca('Sequences'+titles[i], 0)
    
    principalComponents = enc
    plot_pca('Sequences_Not_Pca'+titles[i], 0)

(300, 3)
[0.7992084  0.14315112 0.03062754]


(300, 3)
[0.8448875  0.11824855 0.01743101]


(300, 3)
[0.7992084  0.14315112 0.03062754]
